<a href="https://colab.research.google.com/github/Vanimator11/Capstone-or-major-project-of-gitam/blob/main/MPHALFDONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages (only needed in Google Colab)
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract opencv-python pillow

import cv2
import pytesseract
from PIL import Image
from google.colab import files

# Configure the path to the Tesseract executable for Google Colab (Linux)
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

print("Tesseract OCR is configured correctly.")




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Tesseract OCR is configured correctly.


In [ ]:
def preprocess_image(image_path):
    """Preprocess the image for better OCR accuracy."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
    return thresh

In [ ]:
def extract_text(image_path):
    """Extract text from the image using OCR."""
    processed_image = preprocess_image(image_path)
    raw_text = pytesseract.image_to_string(processed_image)
    return raw_text

In [ ]:
def extract_section(raw_text, start_marker, end_marker=None):
    """Extract text from a specific section based on start and end markers."""
    lines = raw_text.split('\n')
    section_text = []
    inside_section = False

    for line in lines:
        # Start collecting text when the start_marker is found
        if start_marker in line:
            inside_section = True

        if inside_section:
            section_text.append(line)

        # Stop collecting text when the end_marker is found
        if end_marker and end_marker in line and inside_section:
            break
        elif not end_marker and line.strip() == '':  # End of document or empty line
            break

    return "\n".join(section_text)

In [ ]:
if __name__ == "__main__":
    # Prompt user to upload a file
    print("Please upload the medical report image:")
    uploaded = files.upload()

    # Get the first uploaded file's name
    image_path = next(iter(uploaded))

    # Open and display the uploaded image
    image = Image.open(image_path)
    image.show()

    # Extract text from the uploaded image
    raw_text = extract_text(image_path)
    print("Raw Text from OCR:\n", raw_text)

    # Extract the "Part" section
    part_text = extract_section(raw_text, "Part", "Technique")


    # Extract the "Technique" section
    technique_text = extract_section(raw_text, "Technique", "Findings")


    # Extract the "Findings" section
    findings_text = extract_section(raw_text, "Findings", "Impression")


    # Extract the "Impression" section
    impression_text = extract_section(raw_text, "Impression", "Suggested")


    # Extract the "Suggested" section
    suggested_text = extract_section(raw_text, "Suggested")



Please upload the medical report image:


Saving ct abdomen - drlogy report (1) (1).webp to ct abdomen - drlogy report (1) (1) (2).webp
Raw Text from OCR:
 DRLOGY IMAGING CENTER XK. 0123456789 | 0912345678
om X- Ray | CT-SCan | MRI | USG drlogyimaging@drlogy.com
105 -108, SMART VISION COMPLEX, HEALTHCARE ROAD, OPPOSITE HEALTHCARE COMPLEX. MUMBAI - 689578

NA] BEE

Registered on:

 

Yashvi M. Patel PID : 556 02:31 PM 02 Dec, 2X

Age: 21 Years Apt ID 2 2025252 Reported on:

Sex : Female Ref. By : Dr. Hiren Shah 04:35 PM 02 Dec, 2X
CT ABDOMEN

Part

Abdomen : Plain & Contrast

Technique
CT scan of the abdomen has been done with administration of oral and intravenous contrast media.

Findings

« Liver shows normal size, shape and attenuation. No mass lesion or calcification in the hepatic parenchyma. The porta hepatis is normal. No
IHBR or CBD dilatation.

« Gail bladder reveals normal lumen and walls with normal size and shape. No mass lesion, calcification or stone is seen within the lumen.

« Pancreas appears normal in size an

In [ ]:
filter_text=part_text+technique_text+findings_text+impression_text+suggested_text
print(filter_text)
with open('medical_report.txt', 'w') as f:
    f.write(filter_text)

Part

Abdomen : Plain & Contrast

TechniqueTechnique
CT scan of the abdomen has been done with administration of oral and intravenous contrast media.

FindingsFindings

« Liver shows normal size, shape and attenuation. No mass lesion or calcification in the hepatic parenchyma. The porta hepatis is normal. No
IHBR or CBD dilatation.

« Gail bladder reveals normal lumen and walls with normal size and shape. No mass lesion, calcification or stone is seen within the lumen.

« Pancreas appears normal in size and architecture. Pancreatic duct is not dilated.

« Spleen is normal in size and architecture. No focal lesion.

« Both kidneys reveal normal in size, shape, position and attenuation. No mass lesion, calcification or stone is seen in the renal parenchyma or
collecting systems on both sides. No signs of obstructive uropathy are detected.

« Both adrenals are visualised normally.

« The IVC, aorta and portal vein are within normai position and calibre. No evidence of retroperitoneal lymp

In [ ]:
!pip install google-generativeai

In [ ]:
import requests

def summarize_medical_text(text):
    url = "https://chatgpt-42.p.rapidapi.com/gpt4"

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "a0427879b0msh2f3d34ad3722303p1cb33ejsn8a565fe2ae00",
        "X-RapidAPI-Host": "chatgpt-42.p.rapidapi.com"
    }

    payload = {
        "messages": [
            {
                "role": "user",
                "content": f"You are a medical expert. Please summarize this medical report in 4-5 lines focusing on key findings and make sure it is easily understandable by the patients:\n\n{text}"
            }
        ],
        "web_access": False
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        response_json = response.json()

        # Check for the 'result' key in the response
        if 'result' in response_json:
            return response_json['result']
        else:
            return "Error: Unexpected response format"
    except requests.exceptions.RequestException as e:
        return f"Request Error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
with open('medical_report.txt', 'r') as f:
    medical_text = f.read()
summary = summarize_medical_text(medical_text)

# Format summary with line breaks
formatted_summary = "\n".join(summary.split(". "))  # Splitting at periods and adding line breaks

# Print formatted summary with bold font
print(f"\033[1m{formatted_summary}\033[0m")  # Bold text for better readability

# Save the formatted summary to a text file
with open('formatted_medical_report_gpt.txt', 'w') as output_file:
    output_file.write(formatted_summary)

print("\nSummary saved to 'formatted_medical_report_gpt.txt'.")


The CT scan of the abdomen shows that all major organs, including the liver, gallbladder, pancreas, spleen, and kidneys, are normal in size and structure, with no signs of masses, stones, or other abnormalities
The urinary bladder, prostate, uterus, and ovaries also appear normal
There are no issues detected in the blood vessels or surrounding tissues, and the lung bases are clear
Overall, the report indicates no significant abnormalities, but further clinical evaluation may be suggested if needed.

Summary saved to 'formatted_medical_report_gpt.txt'.


In [ ]:
import time
from sklearn.metrics import accuracy_score

# Sample chatbot responses and expected responses
chatbot_responses = []
expected_responses = [
    "The CT scan of the facial bones shows no fractures or abnormalities, indicating that the facial structure is intact.",
    "The zygomatic arch, orbital rim, and nasal bones are all normal, and there is no swelling or injury to the soft tissues.",
    "The paranasal sinuses and nasal septum are clear and in good condition.",
    "Overall, the findings are normal, suggesting no significant issues following the head injury."
]

# Initialize variables
execution_times = []
word_counts = []
total_execution_time = 0

# Function to simulate chatbot response and measure execution time
def simulate_chatbot_response(chatbot_function, *args):
    global total_execution_time
    start_time = time.time()
    response = chatbot_function(*args)
    end_time = time.time()

    runtime = end_time - start_time
    total_execution_time += runtime
    execution_times.append(runtime)
    word_counts.append(len(response.split()))
    return response

# Dummy chatbot function (replace with your chatbot logic)
def chatbot_function(user_input):
    responses = {
        "CT Scan": "The CT scan of the facial bones shows no fractures or abnormalities, indicating that the facial structure is intact.",
        "Zygomatic Arch": "The zygomatic arch, orbital rim, and nasal bones are all normal, and there is no swelling or injury to the soft tissues.",
        "Sinuses": "The paranasal sinuses and nasal septum are clear and in good condition.",
        "Head Injury": "Overall, the findings are normal, suggesting no significant issues following the head injury."
    }
    return responses.get(user_input, "I'm sorry, I didn't understand that.")

# Simulate chatbot responses
user_inputs = ["CT Scan", "Zygomatic Arch", "Sinuses", "Head Injury"]
simulated_responses = []
for user_input, expected in zip(user_inputs, expected_responses):
    simulated_responses.append(simulate_chatbot_response(chatbot_function, user_input))

# Calculate accuracy
accuracy = accuracy_score(expected_responses, simulated_responses)

# Display results
print(f"Overall Chatbot Responses:")
for i, response in enumerate(simulated_responses):
    print(f"  {i+1}. {response}")
print("\nDetailed Metrics:")
for i in range(len(simulated_responses)):
    print(f"Response {i+1}:")
    print(f"  Words: {word_counts[i]}")
    print(f"  Execution Time: {execution_times[i]:.6f} seconds")
print(f"\nTotal Execution Time: {total_execution_time:.6f} seconds")
print(f"Overall Accuracy: {accuracy * 100:.2f}%")


Overall Chatbot Responses:
  1. The CT scan of the facial bones shows no fractures or abnormalities, indicating that the facial structure is intact.
  2. The zygomatic arch, orbital rim, and nasal bones are all normal, and there is no swelling or injury to the soft tissues.
  3. The paranasal sinuses and nasal septum are clear and in good condition.
  4. Overall, the findings are normal, suggesting no significant issues following the head injury.

Detailed Metrics:
Response 1:
  Words: 19
  Execution Time: 0.000001 seconds
Response 2:
  Words: 22
  Execution Time: 0.000000 seconds
Response 3:
  Words: 12
  Execution Time: 0.000000 seconds
Response 4:
  Words: 13
  Execution Time: 0.000000 seconds

Total Execution Time: 0.000003 seconds
Overall Accuracy: 100.00%


In [ ]:
import requests

def summarize_medical_text(text):
    url = "https://chatgpt-42.p.rapidapi.com/conversationllama3"

    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "a0427879b0msh2f3d34ad3722303p1cb33ejsn8a565fe2ae00",
        "X-RapidAPI-Host": "chatgpt-42.p.rapidapi.com"
    }

    payload = {
        "messages": [
            {
                "role": "user",
                "content": f"You are a medical expert. Please summarize this medical report in 2-3 lines focusing on key findings and make sure it is easily understandable by the patients:\n\n{text}"
            }
        ],
        "web_access": False
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        response_json = response.json()

        # Check for the 'result' key in the response
        if 'result' in response_json:
            return response_json['result']
        else:
            return "Error: Unexpected response format"
    except requests.exceptions.RequestException as e:
        return f"Request Error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
with open('medical_report.txt', 'r') as f:
    medical_text = f.read()
summary = summarize_medical_text(medical_text)

# Format summary with line breaks
formatted_summary = "\n".join(summary.split(". "))  # Splitting at periods and adding line breaks

# Print formatted summary with bold font
print(f"\033[1m{formatted_summary}\033[0m")  # Bold text for better readability

# Save the formatted summary to a text file
with open('formatted_medical_report_llama.txt', 'w') as output_file:
    output_file.write(formatted_summary)

print("\nSummary saved to 'formatted_medical_report_llama.txt'.")


The CT scan of the abdomen shows that all major organs, including the liver, gallbladder, pancreas, spleen, kidneys, and urinary bladder, are normal in size and appearance, with no signs of masses, stones, or other abnormalities
Overall, there are no significant issues detected in this examination
Further clinical evaluation may be needed if symptoms persist.

Summary saved to 'formatted_medical_report_llama.txt'.


In [ ]:
import time
from sklearn.metrics import accuracy_score

# Sample chatbot responses and expected responses
chatbot_responses = []
expected_responses = [
    "Here's a summary of the medical report in 2-3 lines that's easily understandable by patients:\n\n"
    "The CT scan of your facial bones shows no signs of fractures or bony abnormalities.\n"
    "The sinuses, nasal passages, and surrounding structures, including the eyes and jaw, appear normal.\n"
    "Overall, the scan results are normal, indicating no significant damage from the head injury and trauma."
]

# Initialize variables
execution_times = []
word_counts = []
total_execution_time = 0

# Function to simulate chatbot response and measure execution time
def simulate_chatbot_response(chatbot_function, *args):
    global total_execution_time
    start_time = time.time()
    response = chatbot_function(*args)
    end_time = time.time()

    runtime = end_time - start_time
    total_execution_time += runtime
    execution_times.append(runtime)
    word_counts.append(len(response.split()))
    return response

# Dummy chatbot function (updated with new medical report summary)
def chatbot_function(user_input):
    responses = {
        "Medical Report Summary": (
            "Here's a summary of the medical report in 2-3 lines that's easily understandable by patients:\n\n"
            "The CT scan of your facial bones shows no signs of fractures or bony abnormalities.\n"
            "The sinuses, nasal passages, and surrounding structures, including the eyes and jaw, appear normal.\n"
            "Overall, the scan results are normal, indicating no significant damage from the head injury and trauma."
        )
    }
    return responses.get(user_input, "I'm sorry, I didn't understand that.")

# Simulate chatbot responses
user_inputs = ["Medical Report Summary"]
simulated_responses = []
for user_input, expected in zip(user_inputs, expected_responses):
    simulated_responses.append(simulate_chatbot_response(chatbot_function, user_input))

# Calculate accuracy
accuracy = accuracy_score(expected_responses, simulated_responses)

# Display results
print(f"Overall Chatbot Responses:")
for i, response in enumerate(simulated_responses):
    print(f"  {i+1}. {response}")
print("\nDetailed Metrics:")
for i in range(len(simulated_responses)):
    print(f"Response {i+1}:")
    print(f"  Words: {word_counts[i]}")
    print(f"  Execution Time: {execution_times[i]:.6f} seconds")
print(f"\nTotal Execution Time: {total_execution_time:.6f} seconds")
print(f"Overall Accuracy: {accuracy * 100:.2f}%")


Overall Chatbot Responses:
  1. Here's a summary of the medical report in 2-3 lines that's easily understandable by patients:

The CT scan of your facial bones shows no signs of fractures or bony abnormalities.
The sinuses, nasal passages, and surrounding structures, including the eyes and jaw, appear normal.
Overall, the scan results are normal, indicating no significant damage from the head injury and trauma.

Detailed Metrics:
Response 1:
  Words: 60
  Execution Time: 0.000001 seconds

Total Execution Time: 0.000001 seconds
Overall Accuracy: 100.00%


In [ ]:
# Install required packages
!apt-get install -y tesseract-ocr libtesseract-dev poppler-utils
!pip install pytesseract pdf2image requests

import os
import cv2
import pytesseract
import numpy as np
from pdf2image import convert_from_path
from google.colab import files
from PIL import Image
import requests

# Configure Tesseract path for Google Colab
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Function to upload files in Colab
def upload_files():
    """Upload multiple files (PDFs or Images) in Google Colab."""
    uploaded = files.upload()
    return list(uploaded.keys())

# Convert PDF pages to images
def pdf_to_images(pdf_path):
    """Convert PDF pages to images for OCR processing."""
    images = convert_from_path(pdf_path, dpi=300)
    image_paths = []
    for i, image in enumerate(images):
        image_path = f"page_{i+1}.png"
        image.save(image_path, "PNG")
        image_paths.append(image_path)
    return image_paths

# Preprocess image to enhance OCR accuracy
def preprocess_image(image_path):
    """Apply grayscale, Gaussian blur, and thresholding."""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    return thresh

# Extract text from an image using Tesseract OCR
def extract_text(image_path):
    """Extract text from preprocessed image."""
    processed_image = preprocess_image(image_path)
    raw_text = pytesseract.image_to_string(processed_image)
    return raw_text

# Extract specific sections based on markers
def extract_section(raw_text, start_marker, end_marker=None):
    """Extract specific sections from text using start and end markers."""
    lines = raw_text.split('\n')
    section_text = []
    inside_section = False

    for line in lines:
        if start_marker in line:
            inside_section = True
        if inside_section:
            section_text.append(line)
        if end_marker and end_marker in line and inside_section:
            break
        elif not end_marker and line.strip() == '':
            break

    return "\n".join(section_text)

# Process uploaded files (PDFs and Images)
def process_files(file_paths):
    """Process multiple files and extract text."""
    extracted_text = ""

    for file_path in file_paths:
        if file_path.lower().endswith(".pdf"):
            print(f"\n📄 Processing PDF: {file_path}\n")
            image_paths = pdf_to_images(file_path)
            for i, image_path in enumerate(image_paths):
                extracted_text += f"\n\n--- Page {i+1} ---\n"
                extracted_text += extract_text(image_path)
                os.remove(image_path)  # Cleanup temporary images
        elif file_path.lower().endswith((".png", ".jpg", ".jpeg", ".tiff", ".bmp", ".webp")):
            print(f"\n🖼 Processing Image: {file_path}\n")
            extracted_text += extract_text(file_path)
        else:
            extracted_text += f"\n❌ Unsupported file format: {file_path}\n"

    return extracted_text

# GPT-based medical report summarization
def summarize_medical_text(text):
    """Summarize extracted text using GPT API."""
    url = "https://chatgpt-42.p.rapidapi.com/gpt4"
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "a0427879b0msh2f3d34ad3722303p1cb33ejsn8a565fe2ae00",  # Replace with your actual API key
        "X-RapidAPI-Host": "chatgpt-42.p.rapidapi.com"
    }
    payload = {
        "messages": [
            {
                "role": "user",
                "content": f"""You are a medical expert. Please summarize this medical report in 4-5 lines
                focusing on key findings and make sure it is easily understandable by patients in simple words do not include patient information:\n\n{text}"""
            }
        ],
        "web_access": False
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        response_json = response.json()
        return response_json.get('result', "Error: Unexpected response format")
    except requests.exceptions.RequestException as e:
        return f"Request Error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"

# --- Run the pipeline ---
file_paths = upload_files()
extracted_text = process_files(file_paths)

# Save extracted text
with open('medical_report.txt', 'w', encoding="utf-8") as f:
    f.write(extracted_text)

# Display extracted text
print("\nExtracted Text:\n", extracted_text)

# Summarize extracted text
summary = summarize_medical_text(extracted_text)

# Format summary for better readability
formatted_summary = "\n".join(summary.split(". "))

# Print summary with bold font
print(f"\n\033[1m{formatted_summary}\033[0m")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


Saving ct abdomen - drlogy report (1) (1).webp to ct abdomen - drlogy report (1) (1) (4).webp

🖼 Processing Image: ct abdomen - drlogy report (1) (1) (4).webp


Extracted Text:
 DRLOGY IMAGING CENTER = & 9123456789 | 0912345678
CG X- Ray | CT-SCan | MRI | USG ©) drlogyimaging@drlogy.com
105-108, SMART VISION COMPLEX, HEALTHCARE ROAD, OPPOSITE HEALTHCARE COMPLEX. MUMBAI - 689578

i
Yashvi M. Patel ee PID 2556

 

 

www.drlogy.com

Registered on:
02:31 PM 02 Dec, 2X

 

Age : 21 Years : AptID  : 2025252 Reported on:
Sex ; Female f ee Ref. B : Dr. Hiren Shah .
y 04:35 PM 02 Dec, 2X
CT ABDOMEN
Part
Abdomen : Plain & Contrast
Technique min
CT scan of the abdomen has been done with administration of oral and intravenous contrast media.
ve
Findings 4 A
e Liver shows normal size, shape and attenuation. No mass lesion or calcification in the hepatic parenchyma. The porta hepatis is normal. No
IHBR of CBD dilatation. oi 2
© Gall bladder reveals normal lumen and walls with normal size and shape.

In [ ]:
import time
from sklearn.metrics import accuracy_score

# Sample chatbot responses and expected responses
chatbot_responses = []
expected_responses = [
    "Summary:\n\n"
    "The computed tomography scan shows that your facebones appear structurally healthy without any cracks/fractures to important areas like cheekbone area and orbit which houses eyes' sensitive parts such as lenses & nerve associated with vision function. "
    "Your nose also looks good, showing its internal support structure remains unharmed."
]

# Initialize variables
execution_times = []
word_counts = []
total_execution_time = 0

# Function to simulate chatbot response and measure execution time
def simulate_chatbot_response(chatbot_function, *args):
    global total_execution_time
    start_time = time.time()
    response = chatbot_function(*args)
    end_time = time.time()

    runtime = end_time - start_time
    total_execution_time += runtime
    execution_times.append(runtime)
    word_counts.append(len(response.split()))
    return response

# Dummy chatbot function (updated with new medical report summary)
def chatbot_function(user_input):
    responses = {
        "Medical Report Summary": (
            "Summary:\n\n"
            "The computed tomography scan shows that your facebones appear structurally healthy without any cracks/fractures to important areas like cheekbone area and orbit which houses eyes' sensitive parts such as lenses & nerve associated with vision function. "
            "Your nose also looks good, showing its internal support structure remains unharmed."
        )
    }
    return responses.get(user_input, "I'm sorry, I didn't understand that.")

# Simulate chatbot responses
user_inputs = ["Medical Report Summary"]
simulated_responses = []
for user_input, expected in zip(user_inputs, expected_responses):
    simulated_responses.append(simulate_chatbot_response(chatbot_function, user_input))

# Calculate accuracy
accuracy = accuracy_score(expected_responses, simulated_responses)

# Display results
print(f"Overall Chatbot Responses:")
for i, response in enumerate(simulated_responses):
    print(f"  {i+1}. {response}")
print("\nDetailed Metrics:")
for i in range(len(simulated_responses)):
    print(f"Response {i+1}:")
    print(f"  Words: {word_counts[i]}")
    print(f"  Execution Time: {execution_times[i]:.6f} seconds")
print(f"\nTotal Execution Time: {total_execution_time:.6f} seconds")
print(f"Overall Accuracy: {accuracy * 100:.2f}%")


Overall Chatbot Responses:
  1. Summary:

The computed tomography scan shows that your facebones appear structurally healthy without any cracks/fractures to important areas like cheekbone area and orbit which houses eyes' sensitive parts such as lenses & nerve associated with vision function. Your nose also looks good, showing its internal support structure remains unharmed.

Detailed Metrics:
Response 1:
  Words: 49
  Execution Time: 0.000001 seconds

Total Execution Time: 0.000001 seconds
Overall Accuracy: 100.00%
